# ME Model Metabolites

## 1. Load Python Packages

In [1]:
from __future__ import print_function, division, absolute_import

# python imports
import re
from os.path import join
from collections import defaultdict
import pickle
import pandas as pd
from pandas import DataFrame

# third party imports
import pandas
import tabulate
import cobra
pd.set_option('display.max_columns', 100)
pd.set_option('display.width',100)
pd.set_option('display.max_colwidth',100)

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
#from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

C:\Users\a00016187\.conda\envs\COBRAme\lib\site-packages\cobra\io\sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
C:\Users\a00016187\.conda\envs\COBRAme\lib\site-packages\cobra\io\__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


## 2. Load the iJL1678b model

Load the iJL1678b model in pickle format that is located in the current directory.


In [2]:
with open('./me_models/iJL1678b.pickle', 'rb') as f:
    me = pickle.load(f)

List the attributes of the iJL1678b mode

In [3]:
pd.DataFrame({i: str(v) for i, v in me.__dict__.items()}, 
             index=['Attribute Values']).T

,Attribute Values
id,iJL1678b-ME
name,None
notes,{}
annotation,{}
_trimmed,False
_trimmed_genes,[]
_trimmed_reactions,{}
genes,[]
reactions,"[<SummaryVariable biomass_dilution at 0x240f85c9470>, <SummaryVariable protein_biomass_to_biomas..."
metabolites,"[<Constraint biomass at 0x2406edb22b0>, <Constraint protein_biomass at 0x2406edb2518>, <Constrai..."


The attributes of the ME model can be listed by attaching the name of the attribute to the name of the ME model preceeded by a "." Below are a couple of examples showing the metabolites, reactions and global-info of the model.

In [4]:
me.metabolites

[<Constraint biomass at 0x2406edb22b0>,
 <Constraint protein_biomass at 0x2406edb2518>,
 <Constraint mRNA_biomass at 0x2406edb25c0>,
 <Constraint tRNA_biomass at 0x2406edb2630>,
 <Constraint rRNA_biomass at 0x2406edb26a0>,
 <Constraint ncRNA_biomass at 0x2406edb27f0>,
 <Constraint DNA_biomass at 0x240f9c98ac8>,
 <Constraint lipid_biomass at 0x240f9c98b38>,
 <Constraint constituent_biomass at 0x240f9cb6080>,
 <Constraint prosthetic_group_biomass at 0x240f9cf3208>,
 <Constraint peptidoglycan_biomass at 0x240f9ec8128>,
 <Metabolite 10fthf_c at 0x240fbb91f60>,
 <Metabolite 12dgr120_c at 0x2406969c668>,
 <Metabolite 12dgr140_c at 0x2406969c7f0>,
 <Metabolite 12dgr141_c at 0x2406969c908>,
 <Metabolite 12dgr160_c at 0x2406969ca58>,
 <Metabolite 12dgr161_c at 0x2406969cb70>,
 <Metabolite 12dgr180_c at 0x2406969cc50>,
 <Metabolite 12dgr181_c at 0x2406969cd68>,
 <Metabolite 12ppd__R_c at 0x240696bd470>,
 <Metabolite 12ppd__S_c at 0x240696bd828>,
 <Metabolite 13dpg_c at 0x24069e08f60>,
 <Metaboli

In [5]:
metabolite_names = [m.name for m in me.metabolites]
metabolite_ids = [m.id for m in me.metabolites]
metabolite_compartment = [m.compartment for m in me.metabolites]
metabolite_formula = [m.formula for m in me.metabolites]
metabolite_charge = [m.charge for m in me.metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Name': metabolite_names,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Charge': metabolite_charge
                 }

pd.set_option('display.max_rows', 500)
df = pd.DataFrame(metaboliteList, columns= ['Metabolite ID','Metabolite Name', 'Metabolite Formula', 'Metabolite Compartment','Metabolite Charge'])
df[0:200]

,Metabolite ID,Metabolite Name,Metabolite Formula,Metabolite Compartment,Metabolite Charge
0,biomass,,None,mc,NaN
1,protein_biomass,,None,mc,NaN
2,mRNA_biomass,,None,mc,NaN
3,tRNA_biomass,,None,mc,NaN
4,rRNA_biomass,,None,mc,NaN
5,ncRNA_biomass,,None,mc,NaN
6,DNA_biomass,,None,mc,NaN
7,lipid_biomass,,None,mc,NaN
8,constituent_biomass,,None,mc,NaN
9,prosthetic_group_biomass,,None,mc,NaN


In [6]:
me.reactions

[<SummaryVariable biomass_dilution at 0x240f85c9470>,
 <SummaryVariable protein_biomass_to_biomass at 0x2406edb24e0>,
 <SummaryVariable mRNA_biomass_to_biomass at 0x2406edb2588>,
 <SummaryVariable tRNA_biomass_to_biomass at 0x2406edb25f8>,
 <SummaryVariable rRNA_biomass_to_biomass at 0x2406edb2668>,
 <SummaryVariable ncRNA_biomass_to_biomass at 0x2406edb26d8>,
 <SummaryVariable DNA_biomass_to_biomass at 0x2406edb2748>,
 <SummaryVariable lipid_biomass_to_biomass at 0x240f9c98b00>,
 <SummaryVariable constituent_biomass_to_biomass at 0x240f9d7bf98>,
 <SummaryVariable prosthetic_group_biomass_to_biomass at 0x240f9cb60b8>,
 <SummaryVariable peptidoglycan_biomass_to_biomass at 0x240f9ec80f0>,
 <MEReaction DM_4CRSOL at 0x240f9ec8160>,
 <MEReaction DM_5DRIB at 0x240f9ec8208>,
 <MEReaction DM_AACALD at 0x240f9ec82e8>,
 <MEReaction DM_AMOB at 0x240f9ec8390>,
 <MEReaction DM_MTHTHF at 0x240f9ec8470>,
 <MEReaction DM_OXAM at 0x240f9ec8518>,
 <MEReaction EX_12ppd__R_e at 0x240f9ec8668>,
 <MEReactio

In [7]:
me.global_info

{'kt': 4.5,
 'r0': 0.087,
 'k_deg': 12.0,
 'm_rr': 1453.0,
 'm_aa': 0.109,
 'f_rRNA': 0.86,
 'm_nt': 0.324,
 'f_mRNA': 0.02,
 'm_tRNA': 25.0,
 'f_tRNA': 0.12,
 'temperature': 37,
 'propensity_scaling': 0.45,
 'GC_fraction': 0.507896997096}

In [8]:
met = me.metabolites.get_by_id('RNAPS-CPLX')
pd.DataFrame({i: str(v) for i, v in met.__dict__.items()}, index=['Attribute Values']).T

,Attribute Values
id,RNAPS-CPLX
name,
notes,{}
annotation,{}
_model,iJL1678b-ME
_reaction,"{<TranscriptionReaction transcription_TU0_2661_from_RPOS_MONOMER at 0x240fa894048>, <Transcripti..."
formula,C18332H29619Mg2N5166O5657S118Zn
compartment,c
charge,None
_constraint_sense,E


## 3. Attributes of metabolites

COBRAme supports 10 types of metabolites, they include:

1. GenericComponent
2. ProcessedProtein
3. GenerictRNA
4. Constraint
5. Complex
6. Metabolite
7. RNAP
8. Ribosome
9. TranscribedGene
10. TranslatedGene

Most metabolites effectively behave the same as a typical COBRA metabolite. The main exception is the TranscribedGene metaboltie type which contains additional sequence information about the RNA it represents. An additional metabolite_involvement column is present which lists the types of MEReactions that each metabolite participates in. The compartments correspond to:  "p"="Periplasm", "e"= "Extra-organism" ,  "c"="Cytosol",   "im"= 'Inner Membrane ", "om"= "Outer Membrane", "mc"="ME-model Constraint",  "m"="Membrane"!

For the more about the metabolite types refer to the online documentation (https://cobrame.readthedocs.io) and look at the ME metabolites table (S7) included in the supplementary material (https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1006302)

### "GenericComponent" Metabolites

The attributes and methods of the "GenericComponent" Metabolites are

- annotation,
- charge,
- compartment,
- copy,
- elements,
- formula,
- formula_weight,
- id,
- model,
- name,
- notes,
- reactions,
- remove_from_me_model,
- remove_from_model,
- summary,
- y

A table showing the key attributes of "GenericComponent" Metabolites can be found by

In [9]:
generic_component_metabolites = []
GenericComponent_metabolites = [m.id for m in me.metabolites if type(m) == cobrame.GenericComponent]
for m in GenericComponent_metabolites: generic_component_metabolites.append(me.metabolites.get_by_id(m))

metabolite_ids = [m.id for m in generic_component_metabolites]
metabolite_charge = [m.charge for m in generic_component_metabolites]
metabolite_formula = [m.formula for m in generic_component_metabolites]
metabolite_formula_weight = [m.formula_weight for m in generic_component_metabolites]
metabolite_compartment = [m.compartment for m in generic_component_metabolites]
metabolite_elements = [m.elements for m in generic_component_metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Formula Weight': metabolite_formula_weight,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Elements': metabolite_elements,
                 }

TranslatedGene_Metabolites = pd.DataFrame(metaboliteList, columns= ['Metabolite ID', 'Metabolite Formula', 'Metabolite Formula Weight','Metabolite Compartment', 'Metabolite Elements'])
pd.set_option('display.max_rows', 500)
TranslatedGene_Metabolites[0:200] 

,Metabolite ID,Metabolite Formula,Metabolite Formula Weight,Metabolite Compartment,Metabolite Elements
0,generic_16Sm4Cm1402,None,0,c,{}
1,generic_LYSINEaaRS,None,0,c,{}
2,generic_Dus,None,0,c,{}
3,generic_RF,None,0,c,{}
4,generic_Tuf,None,0,c,{}
5,generic_RNase,None,0,c,{}
6,generic_16s_rRNAs,None,0,c,{}
7,generic_23s_rRNAs,None,0,c,{}
8,generic_5s_rRNAs,None,0,c,{}
9,generic_2fe2s_transfer_complex,None,0,c,{}


In [10]:
dir(me.metabolites.get_by_id('generic_16Sm4Cm1402'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bound',
 '_constraint_sense',
 '_model',
 '_reaction',
 'annotation',
 'charge',
 'compartment',
 'copy',
 'elements',
 'formula',
 'formula_weight',
 'id',
 'model',
 'name',
 'notes',
 'reactions',
 'remove_from_me_model',
 'remove_from_model',
 'summary',
 'y']

### "ProcessedProtein" Metabolites

The attributes and methods of the "ProcessedProtein" Metabolites are

- annotation,
- charge,
- compartment,
- copy,
- elements,
- formula,
- formula_weight,
- id,
- model,
- name,
- notes,
- reactions,
- remove_from_me_model,
- remove_from_model,
- summary,
- unprocessed_protein,
- unprocessed_protein_id,
- y

A table showing the key attributes of "ProcessedProtein" Metabolites can be found by

In [11]:
processed_protein_metabolites = []
ProcessedProtein_metabolites = [m.id for m in me.metabolites if type(m) == cobrame.ProcessedProtein]
for m in ProcessedProtein_metabolites: processed_protein_metabolites.append(me.metabolites.get_by_id(m))

metabolite_ids = [m.id for m in processed_protein_metabolites]
metabolite_charge = [m.charge for m in processed_protein_metabolites]
metabolite_formula = [m.formula for m in processed_protein_metabolites]
metabolite_formula_weight = [m.formula_weight for m in processed_protein_metabolites]
metabolite_compartment = [m.compartment for m in processed_protein_metabolites]
metabolite_elements = [m.elements for m in processed_protein_metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Formula Weight': metabolite_formula_weight,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Elements': metabolite_elements,
                 }

TranslatedGene_Metabolites = pd.DataFrame(metaboliteList, columns= ['Metabolite ID', 'Metabolite Formula', 'Metabolite Formula Weight','Metabolite Compartment', 'Metabolite Elements'])
pd.set_option('display.max_rows', 500)
TranslatedGene_Metabolites[0:200] 

,Metabolite ID,Metabolite Formula,Metabolite Formula Weight,Metabolite Compartment,Metabolite Elements
0,protein_b0007_Inner_Membrane,C2391H3689N602O619S28,51669.35636,im,"{'C': 2391, 'H': 3689, 'N': 602, 'O': 619, 'S': 28}"
1,protein_b0019_Inner_Membrane,C1937H3109N480O487S14,41362.24516,im,"{'C': 1937, 'H': 3109, 'N': 480, 'O': 487, 'S': 14}"
2,protein_b0027_Inner_Membrane,C853H1300N211O216S6,18159.12320,im,"{'C': 853, 'H': 1300, 'N': 211, 'O': 216, 'S': 6}"
3,protein_b0040_Inner_Membrane,C2683H4070N622O664S29,56592.67790,im,"{'C': 2683, 'H': 4070, 'N': 622, 'O': 664, 'S': 29}"
4,protein_b0047_Inner_Membrane,C3071H4943N812O849S30,67785.98812,im,"{'C': 3071, 'H': 4943, 'N': 812, 'O': 849, 'S': 30}"
5,protein_b0053_Periplasm,C2054H3333N608O639S16,47282.17202,p,"{'C': 2054, 'H': 3333, 'N': 608, 'O': 639, 'S': 16}"
6,protein_b0067_Inner_Membrane,C2769H4399N708O694S26,59545.57356,im,"{'C': 2769, 'H': 4399, 'N': 708, 'O': 694, 'S': 26}"
7,protein_b0068_Inner_Membrane,C1651H2543N420O475S9,36163.97112,im,"{'C': 1651, 'H': 2543, 'N': 420, 'O': 475, 'S': 9}"
8,protein_b0070_Inner_Membrane,C1977H3108N488O501S31,42722.81542,im,"{'C': 1977, 'H': 3108, 'N': 488, 'O': 501, 'S': 31}"
9,protein_b0084_Inner_Membrane,C2840H4588N806O833S17,63896.82212,im,"{'C': 2840, 'H': 4588, 'N': 806, 'O': 833, 'S': 17}"


The unprocessed protein for this specific metabolite is

In [12]:
me.metabolites.get_by_id('protein_b0007_Inner_Membrane').unprocessed_protein

<TranslatedGene protein_b0007 at 0x240fbbbb4e0>

The id for the unprocessed protein for this specific metabolite is

In [13]:
me.metabolites.get_by_id('protein_b0007_Inner_Membrane').unprocessed_protein_id

'protein_b0007'

An overview of the methods and attributes of the "ProcessedProtein" Metabolites

In [14]:
dir(me.metabolites.get_by_id('protein_b0007_Inner_Membrane'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bound',
 '_constraint_sense',
 '_model',
 '_reaction',
 'annotation',
 'charge',
 'compartment',
 'copy',
 'elements',
 'formula',
 'formula_weight',
 'id',
 'model',
 'name',
 'notes',
 'reactions',
 'remove_from_me_model',
 'remove_from_model',
 'summary',
 'unprocessed_protein',
 'unprocessed_protein_id',
 'y']

### "GenerictRNA" Metabolites

The attributes and methods of the "GenerictRNA" Metabolites are

- annotation,
- charge,
- compartment,
- copy,
- elements,
- formula,
- formula_weight,
- id,
- model,
- name,
- notes,
- reactions,
- remove_from_me_model,
- remove_from_model,
- summary,
- y

A table showing the key attributes of "GenerictRNA" Metabolites can be found by

In [15]:
generic_tRNA_metabolites = []
GenerictRNA_metabolites = [m.id for m in me.metabolites if type(m) == cobrame.GenerictRNA]
for m in GenerictRNA_metabolites: generic_tRNA_metabolites.append(me.metabolites.get_by_id(m))

metabolite_ids = [m.id for m in generic_tRNA_metabolites]
metabolite_charge = [m.charge for m in generic_tRNA_metabolites]
metabolite_formula = [m.formula for m in generic_tRNA_metabolites]
metabolite_formula_weight = [m.formula_weight for m in generic_tRNA_metabolites]
metabolite_compartment = [m.compartment for m in generic_tRNA_metabolites]
metabolite_elements = [m.elements for m in generic_tRNA_metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Formula Weight': metabolite_formula_weight,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Elements': metabolite_elements,
                 }

TranslatedGene_Metabolites = pd.DataFrame(metaboliteList, columns= ['Metabolite ID', 'Metabolite Formula', 'Metabolite Formula Weight','Metabolite Compartment', 'Metabolite Elements'])
pd.set_option('display.max_rows', 500)
TranslatedGene_Metabolites[0:200] 

,Metabolite ID,Metabolite Formula,Metabolite Formula Weight,Metabolite Compartment,Metabolite Elements
0,generic_tRNA_AUC_ile__L_c,None,0,c,{}
1,generic_tRNA_AUU_ile__L_c,None,0,c,{}
2,generic_tRNA_GCA_ala__L_c,None,0,c,{}
3,generic_tRNA_GCG_ala__L_c,None,0,c,{}
4,generic_tRNA_GCU_ala__L_c,None,0,c,{}
5,generic_tRNA_GAC_asp__L_c,None,0,c,{}
6,generic_tRNA_GAU_asp__L_c,None,0,c,{}
7,generic_tRNA_ACG_thr__L_c,None,0,c,{}
8,generic_tRNA_AGA_arg__L_c,None,0,c,{}
9,generic_tRNA_CAA_gln__L_c,None,0,c,{}


An overview of the methods and attributes of the "GenerictRNA" Metabolites

In [16]:
dir(me.metabolites.get_by_id('generic_tRNA_AUC_ile__L_c'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bound',
 '_constraint_sense',
 '_model',
 '_reaction',
 'annotation',
 'charge',
 'compartment',
 'copy',
 'elements',
 'formula',
 'formula_weight',
 'id',
 'model',
 'name',
 'notes',
 'reactions',
 'remove_from_me_model',
 'remove_from_model',
 'summary',
 'y']

### "Constraint" Metabolites

The attributes and methods of the "Constraint" Metabolites are

- annotation,
- charge,
- compartment,
- copy,
- elements,
- formula,
- formula_weight,
- id,
- model,
- name,
- notes,
- reactions,
- remove_from_me_model,
- remove_from_model,
- summary,
- y

A table showing the key attributes of "Constraint" Metabolites can be found by

In [17]:
constraint_based_metabolites = []
Constraint_metabolites = [m.id for m in me.metabolites if type(m) == cobrame.Constraint]
for m in Constraint_metabolites: constraint_based_metabolites.append(me.metabolites.get_by_id(m))

metabolite_ids = [m.id for m in constraint_based_metabolites]
metabolite_charge = [m.charge for m in constraint_based_metabolites]
metabolite_formula = [m.formula for m in constraint_based_metabolites]
metabolite_formula_weight = [m.formula_weight for m in constraint_based_metabolites]
metabolite_compartment = [m.compartment for m in constraint_based_metabolites]
metabolite_elements = [m.elements for m in constraint_based_metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Formula Weight': metabolite_formula_weight,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Elements': metabolite_elements,
                 }

TranslatedGene_Metabolites = pd.DataFrame(metaboliteList, columns= ['Metabolite ID', 'Metabolite Formula', 'Metabolite Formula Weight','Metabolite Compartment', 'Metabolite Elements'])
pd.set_option('display.max_rows', 500)
TranslatedGene_Metabolites

,Metabolite ID,Metabolite Formula,Metabolite Formula Weight,Metabolite Compartment,Metabolite Elements
0,biomass,None,0,mc,{}
1,protein_biomass,None,0,mc,{}
2,mRNA_biomass,None,0,mc,{}
3,tRNA_biomass,None,0,mc,{}
4,rRNA_biomass,None,0,mc,{}
5,ncRNA_biomass,None,0,mc,{}
6,DNA_biomass,None,0,mc,{}
7,lipid_biomass,None,0,mc,{}
8,constituent_biomass,None,0,mc,{}
9,prosthetic_group_biomass,None,0,mc,{}


An overview of the methods and attributes of the "Constraint" Metabolites

In [18]:
dir(me.metabolites.get_by_id('biomass'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bound',
 '_constraint_sense',
 '_model',
 '_reaction',
 'annotation',
 'charge',
 'compartment',
 'copy',
 'elements',
 'formula',
 'formula_weight',
 'id',
 'model',
 'name',
 'notes',
 'reactions',
 'remove_from_me_model',
 'remove_from_model',
 'summary',
 'y']

### "Complex" Metabolites

The attributes and methods of the "Complex" Metabolites are

- annotation,
- charge,
- compartment,
- copy,
- elements,
- formula,
- formula_weight,
- id,
- model,
- name,
- notes,
- reactions,
- remove_from_me_model,
- remove_from_model,
- summary,
- y

A table showing the key attributes of "Complex" Metabolites can be found by

In [19]:
complex_based_metabolites = []
Complex_metabolites = [m.id for m in me.metabolites if type(m) == cobrame.Complex]
for m in Complex_metabolites: complex_based_metabolites.append(me.metabolites.get_by_id(m))

metabolite_ids = [m.id for m in complex_based_metabolites]
metabolite_charge = [m.charge for m in complex_based_metabolites]
metabolite_formula = [m.formula for m in complex_based_metabolites]
metabolite_formula_weight = [m.formula_weight for m in complex_based_metabolites]
metabolite_compartment = [m.compartment for m in complex_based_metabolites]
metabolite_elements = [m.elements for m in complex_based_metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Formula Weight': metabolite_formula_weight,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Elements': metabolite_elements,
                 }

TranslatedGene_Metabolites = pd.DataFrame(metaboliteList, columns= ['Metabolite ID', 'Metabolite Formula', 'Metabolite Formula Weight','Metabolite Compartment', 'Metabolite Elements'])
pd.set_option('display.max_rows', 500)
TranslatedGene_Metabolites[0:200]

,Metabolite ID,Metabolite Formula,Metabolite Formula Weight,Metabolite Compartment,Metabolite Elements
0,CPLX0-7817_mod_Oxidized,C1152H1798N296O348S12,25747.156920,c,"{'C': 1152, 'H': 1798, 'N': 296, 'O': 348, 'S': 12}"
1,CPLX0-7817,C1152H1800N296O348S12,25749.172800,c,"{'C': 1152, 'H': 1800, 'N': 296, 'O': 348, 'S': 12}"
2,GLUTAREDOXIN-MONOMER_mod_Oxidized,C429H657N116O134S3,9679.698680,c,"{'C': 429, 'H': 657, 'N': 116, 'O': 134, 'S': 3}"
3,GLUTAREDOXIN-MONOMER,C429H659N116O134S3,9681.714560,c,"{'C': 429, 'H': 659, 'N': 116, 'O': 134, 'S': 3}"
4,GRXC-MONOMER_mod_Oxidized,C394H634N112O121S4,9004.187560,c,"{'C': 394, 'H': 634, 'N': 112, 'O': 121, 'S': 4}"
5,GRXC-MONOMER,C394H636N112O121S4,9006.203440,c,"{'C': 394, 'H': 636, 'N': 112, 'O': 121, 'S': 4}"
6,GRXB-MONOMER,C1093H1748N294O316S9,24351.939420,c,"{'C': 1093, 'H': 1748, 'N': 294, 'O': 316, 'S': 9}"
7,GRXB-MONOMER_mod_Oxidized,C1093H1746N294O316S9,24349.923540,c,"{'C': 1093, 'H': 1746, 'N': 294, 'O': 316, 'S': 9}"
8,EG50003-MONOMER_mod_pan4p_mod_3ha,C383H600N95O143PS2,8918.516561,c,"{'C': 383, 'H': 600, 'N': 95, 'O': 143, 'P': 1, 'S': 2}"
9,EG50003-MONOMER_mod_pan4p_mod_act,C383H598N95O143PS2,8916.500681,c,"{'C': 383, 'H': 598, 'N': 95, 'O': 143, 'P': 1, 'S': 2}"


An overview of the methods and attributes of the "Complex" Metabolites

In [20]:
dir(me.metabolites.get_by_id('CPLX0-7817_mod_Oxidized'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bound',
 '_constraint_sense',
 '_model',
 '_reaction',
 'annotation',
 'charge',
 'compartment',
 'copy',
 'elements',
 'formula',
 'formula_weight',
 'id',
 'metabolic_reactions',
 'model',
 'name',
 'notes',
 'reactions',
 'remove_from_me_model',
 'remove_from_model',
 'summary',
 'y']

### "Metabolite" Metabolites

The attributes and methods of the "Metabolite" Metabolites are

- annotation,
- charge,
- compartment,
- copy,
- elements,
- formula,
- formula_weight,
- id,
- model,
- name,
- notes,
- reactions,
- remove_from_me_model,
- remove_from_model,
- summary,
- y

A table showing the key attributes of "Metabolite" Metabolites can be found by

In [21]:
standard_metabolite_metabolites = []
Metabolite_metabolites = [m.id for m in me.metabolites if type(m) == cobrame.Metabolite]
for m in Metabolite_metabolites: standard_metabolite_metabolites.append(me.metabolites.get_by_id(m))

metabolite_ids = [m.id for m in standard_metabolite_metabolites]
metabolite_charge = [m.charge for m in standard_metabolite_metabolites]
metabolite_formula = [m.formula for m in standard_metabolite_metabolites]
metabolite_formula_weight = [m.formula_weight for m in standard_metabolite_metabolites]
metabolite_compartment = [m.compartment for m in standard_metabolite_metabolites]
metabolite_elements = [m.elements for m in standard_metabolite_metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Formula Weight': metabolite_formula_weight,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Elements': metabolite_elements,
                 }

TranslatedGene_Metabolites = pd.DataFrame(metaboliteList, columns= ['Metabolite ID', 'Metabolite Formula', 'Metabolite Formula Weight','Metabolite Compartment', 'Metabolite Elements'])
pd.set_option('display.max_rows', 500)
TranslatedGene_Metabolites[0:200]

,Metabolite ID,Metabolite Formula,Metabolite Formula Weight,Metabolite Compartment,Metabolite Elements
0,10fthf_c,C20H21N7O7,471.423440,c,"{'C': 20, 'H': 21, 'N': 7, 'O': 7}"
1,12dgr120_c,C27H52O5,456.698780,c,"{'C': 27, 'H': 52, 'O': 5}"
2,12dgr140_c,C31H60O5,512.805100,c,"{'C': 31, 'H': 60, 'O': 5}"
3,12dgr141_c,C31H56O5,508.773340,c,"{'C': 31, 'H': 56, 'O': 5}"
4,12dgr160_c,C35H68O5,568.911420,c,"{'C': 35, 'H': 68, 'O': 5}"
5,12dgr161_c,C35H64O5,564.879660,c,"{'C': 35, 'H': 64, 'O': 5}"
6,12dgr180_c,C39H76O5,625.017740,c,"{'C': 39, 'H': 76, 'O': 5}"
7,12dgr181_c,C39H72O5,620.985980,c,"{'C': 39, 'H': 72, 'O': 5}"
8,12ppd__R_c,C3H8O2,76.094420,c,"{'C': 3, 'H': 8, 'O': 2}"
9,12ppd__S_c,C3H8O2,76.094420,c,"{'C': 3, 'H': 8, 'O': 2}"


An overview of the methods and attributes of the "Metabolite" Metabolites

In [22]:
dir(me.metabolites.get_by_id('10fthf_c'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bound',
 '_constraint_sense',
 '_model',
 '_reaction',
 'annotation',
 'charge',
 'compartment',
 'copy',
 'elements',
 'formula',
 'formula_weight',
 'id',
 'model',
 'name',
 'notes',
 'reactions',
 'remove_from_me_model',
 'remove_from_model',
 'summary',
 'y']

### "RNAP" Metabolites

The attributes and methods of the "RNAP" Metabolites are

- annotation,
- charge,
- compartment,
- copy,
- elements,
- formula,
- formula_weight,
- id,
- model,
- name,
- notes,
- reactions,
- remove_from_me_model,
- remove_from_model,
- summary,
- y

A table showing the key attributes of "RNAP" Metabolites can be found by

In [23]:
standard_RNAP_metabolites = []
RNAP_metabolites = [m.id for m in me.metabolites if type(m) == cobrame.RNAP]
for m in RNAP_metabolites: standard_RNAP_metabolites.append(me.metabolites.get_by_id(m))

metabolite_ids = [m.id for m in standard_RNAP_metabolites]
metabolite_charge = [m.charge for m in standard_RNAP_metabolites]
metabolite_formula = [m.formula for m in standard_RNAP_metabolites]
metabolite_formula_weight = [m.formula_weight for m in standard_RNAP_metabolites]
metabolite_compartment = [m.compartment for m in standard_RNAP_metabolites]
metabolite_elements = [m.elements for m in standard_RNAP_metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Formula Weight': metabolite_formula_weight,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Elements': metabolite_elements,
                 }

TranslatedGene_Metabolites = pd.DataFrame(metaboliteList, columns= ['Metabolite ID', 'Metabolite Formula', 'Metabolite Formula Weight','Metabolite Compartment', 'Metabolite Elements'])
pd.set_option('display.max_rows', 500)
TranslatedGene_Metabolites

,Metabolite ID,Metabolite Formula,Metabolite Formula Weight,Metabolite Compartment,Metabolite Elements
0,RNAP54-CPLX,C19036H30704Mg2N5332O5899S126Zn,432801.86896,c,"{'C': 19036, 'H': 30704, 'Mg': 2, 'N': 5332, 'O': 5899, 'S': 126, 'Zn': 1}"
1,CPLX0-222,C17871H28883Mg2N5039O5508S118Zn,406357.69622,c,"{'C': 17871, 'H': 28883, 'Mg': 2, 'N': 5039, 'O': 5508, 'S': 118, 'Zn': 1}"
2,RNAPS-CPLX,C18332H29619Mg2N5166O5657S118Zn,416799.23426,c,"{'C': 18332, 'H': 29619, 'Mg': 2, 'N': 5166, 'O': 5657, 'S': 118, 'Zn': 1}"
3,RNAP32-CPLX,C18094H29204Mg2N5097O5567S124Zn,411308.37426,c,"{'C': 18094, 'H': 29204, 'Mg': 2, 'N': 5097, 'O': 5567, 'S': 124, 'Zn': 1}"
4,RNAPE-CPLX,C17631H28484Mg2N4954O5424S118Zn,400538.44106,c,"{'C': 17631, 'H': 28484, 'Mg': 2, 'N': 4954, 'O': 5424, 'S': 118, 'Zn': 1}"
5,RNAP70-CPLX,C19713H31813Mg2N5539O6126S141Zn,449063.14402,c,"{'C': 19713, 'H': 31813, 'Mg': 2, 'N': 5539, 'O': 6126, 'S': 141, 'Zn': 1}"
6,CPLX0-221,C17532H28319Mg2N4914O5393S120Zn,398190.95226,c,"{'C': 17532, 'H': 28319, 'Mg': 2, 'N': 4914, 'O': 5393, 'S': 120, 'Zn': 1}"


An overview of the methods and attributes of the "RNAP" Metabolites

In [24]:
dir(me.metabolites.get_by_id('RNAP54-CPLX'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bound',
 '_constraint_sense',
 '_model',
 '_reaction',
 'annotation',
 'charge',
 'compartment',
 'copy',
 'elements',
 'formula',
 'formula_weight',
 'id',
 'metabolic_reactions',
 'model',
 'name',
 'notes',
 'reactions',
 'remove_from_me_model',
 'remove_from_model',
 'summary',
 'y']

### "Ribosome" Metabolites

The attributes and methods of the "Ribosome" Metabolites are

- annotation,
- charge,
- compartment,
- copy,
- elements,
- formula,
- formula_weight,
- id,
- model,
- name,
- notes,
- reactions,
- remove_from_me_model,
- remove_from_model,
- summary,
- y

A table showing the key attributes of "Ribosome" Metabolites can be found by

In [25]:
standard_ribosome_metabolites = []
Ribosome_metabolites = [m.id for m in me.metabolites if type(m) == cobrame.Ribosome]
for m in Ribosome_metabolites: standard_ribosome_metabolites.append(me.metabolites.get_by_id(m))

metabolite_ids = [m.id for m in standard_ribosome_metabolites]
metabolite_charge = [m.charge for m in standard_ribosome_metabolites]
metabolite_formula = [m.formula for m in standard_ribosome_metabolites]
metabolite_formula_weight = [m.formula_weight for m in standard_ribosome_metabolites]
metabolite_compartment = [m.compartment for m in standard_ribosome_metabolites]
metabolite_elements = [m.elements for m in standard_ribosome_metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Formula Weight': metabolite_formula_weight,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Elements': metabolite_elements,
                 }

TranslatedGene_Metabolites = pd.DataFrame(metaboliteList, columns= ['Metabolite ID', 'Metabolite Formula', 'Metabolite Formula Weight','Metabolite Compartment', 'Metabolite Elements'])
pd.set_option('display.max_rows', 500)
TranslatedGene_Metabolites

,Metabolite ID,Metabolite Formula,Metabolite Formula Weight,Metabolite Compartment,Metabolite Elements
0,ribosome,C38487H64147Mg171N11545O11295P3S201,880026.853863,c,"{'C': 38487, 'H': 64147, 'Mg': 171, 'N': 11545, 'O': 11295, 'P': 3, 'S': 201}"


An overview of the methods and attributes of the "Ribosome" Metabolites

In [26]:
dir(me.metabolites.get_by_id('ribosome'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bound',
 '_constraint_sense',
 '_model',
 '_reaction',
 'annotation',
 'charge',
 'compartment',
 'copy',
 'elements',
 'formula',
 'formula_weight',
 'id',
 'metabolic_reactions',
 'model',
 'name',
 'notes',
 'reactions',
 'remove_from_me_model',
 'remove_from_model',
 'summary',
 'y']

### "TranscribedGene" Metabolites
See section 4.2.1. "TranscribedGene (RNA) metabolite properties" at https://cobrame.readthedocs.io/en/master/Reaction_Manipulation.html

The *E. coli* ME-model has 4 possible RNA types that can be transcribed: 
   - mRNA 
   - tRNA 
   - rRNA 
   - ncRNA (noncoding RNA)
   
mRNAs can then translated directly from the full transcribed TU, while rRNA, tRNA and ncRNA are spliced out of the TU by endonucleases. In these cases, in order to know which bases need excized, the RNA metabolites (TranscribedGene) themselves have to store information such as: 
   - DNA strand, left and right genome position to identify which TU the RNA is a part of 
   - RNA type to determine whether it needs excised from the TU 
   - nucleotide sequence to determine bases that do/do not need excised if not mRNA and the RNA mass for biomass constraint

The number of "TranscribedGene" metabolites in the model can be found by


In [27]:
print('Number of "TranscribedGene" metabolites (RNA) in the iJL1678b-ME model = %i' %
      len([m.id for m in me.metabolites if type(m) == cobrame.TranscribedGene]))

Number of "TranscribedGene" metabolites (RNA) in the iJL1678b-ME model = 1679


A table listing all the "TranscribedGene" metabolites can be created as follows

In [28]:
RNA_type_metabolites = []
transcribed_metabolites = [m.id for m in me.metabolites if type(m) == cobrame.TranscribedGene]
for m in transcribed_metabolites: RNA_type_metabolites.append(me.metabolites.get_by_id(m))

metabolite_names = [m.name for m in RNA_type_metabolites]
metabolite_ids = [m.id for m in RNA_type_metabolites]
metabolite_compartment = [m.compartment for m in RNA_type_metabolites]
metabolite_formula = [m.formula for m in RNA_type_metabolites]
metabolite_type = [m.RNA_type for m in RNA_type_metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Name': metabolite_names,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Type': metabolite_type
                 }

TranscribedGene_Metabolites = pd.DataFrame(metaboliteList, columns= ['Metabolite ID','Metabolite Name', 'Metabolite Formula', 'Metabolite Compartment','Metabolite Type'])
pd.set_option('display.max_rows', 500)
TranscribedGene_Metabolites[0:200]   

,Metabolite ID,Metabolite Name,Metabolite Formula,Metabolite Compartment,Metabolite Type
0,RNA_b0002,,C23385H26540N9272O17191P2463,c,mRNA
1,RNA_b0003,,C8841H10068N3492O6513P933,c,mRNA
2,RNA_b0004,,C12201H13852N4792O9012P1287,c,mRNA
3,RNA_b0007,,C13657H15482N5590O9867P1431,c,mRNA
4,RNA_b0008,,C9033H10242N3504O6724P954,c,mRNA
5,RNA_b0009,,C5568H6331N2179O4122P588,c,mRNA
6,RNA_b0014,,C18122H20544N6946O13568P1917,c,mRNA
7,RNA_b0015,,C10688H12165N4135O7962P1131,c,mRNA
8,RNA_b0019,,C11120H12624N4522O8028P1167,c,mRNA
9,RNA_b0023,,C2494H2815N939O1885P264,c,mRNA


#### mRNA Metabolites

A table listing the "TranscribedGene" mRNA Metabolites can be found by

In [29]:
df = TranscribedGene_Metabolites
mrna_metabolites = df.loc[df['Metabolite Type'] == 'mRNA']
print("Number of mRNA metabolites = ",len(mrna_metabolites))
mrna_metabolites

Number of mRNA metabolites =  1569


,Metabolite ID,Metabolite Name,Metabolite Formula,Metabolite Compartment,Metabolite Type
0,RNA_b0002,,C23385H26540N9272O17191P2463,c,mRNA
1,RNA_b0003,,C8841H10068N3492O6513P933,c,mRNA
2,RNA_b0004,,C12201H13852N4792O9012P1287,c,mRNA
3,RNA_b0007,,C13657H15482N5590O9867P1431,c,mRNA
4,RNA_b0008,,C9033H10242N3504O6724P954,c,mRNA
...,...,...,...,...,...
1674,RNA_b4390,,C11674H13250N4540O8652P1233,c,mRNA
1675,RNA_b4392,,C18337H20849N7135O13623P1938,c,mRNA
1676,RNA_b4394,,C4867H5540N1936O3572P513,c,mRNA
1677,RNA_b4395,,C6133H6980N2398O4548P648,c,mRNA


The attributes of a specific "TranscribedGene" mRNA metabolite can be found using

In [30]:
pd.DataFrame({i: str(v) for i, v in me.metabolites.RNA_b4025.__dict__.items() if not i.startswith('_') and v}, 
             index=['Attribute Values']).T

,Attribute Values
id,RNA_b4025
formula,C15653H17725N6131O11597P1650
compartment,c
left_pos,4231780
right_pos,4233430
strand,+
RNA_type,mRNA
nucleotide_sequence,ATGAAAAACATCAATCCAACGCAGACCGCTGCCTGGCAGGCACTACAGAAACACTTCGATGAAATGAAAGACGTTACGATCGCCGATCTTTTTGCT...


Reactions that consume or produce this mRNA metabolite are

In [31]:
me.metabolites.get_by_id('RNA_b4025').reactions

frozenset({<MEReaction DM_RNA_b4025 at 0x240692482b0>,
           <TranscriptionReaction transcription_TU0_6444_from_RPOS_MONOMER at 0x240fa92eda0>,
           <TranscriptionReaction transcription_TU0_6444_from_RpoD_mono at 0x240fa92ef28>,
           <TranslationReaction translation_b4025 at 0x240ffc4f160>})

The specific DNA nucleotide sequence for a given mRNA metabolite can be found by

In [32]:
me.metabolites.get_by_id('RNA_b4025').nucleotide_sequence

'ATGAAAAACATCAATCCAACGCAGACCGCTGCCTGGCAGGCACTACAGAAACACTTCGATGAAATGAAAGACGTTACGATCGCCGATCTTTTTGCTAAAGACGGCGATCGTTTTTCTAAGTTCTCCGCAACCTTCGACGATCAGATGCTGGTGGATTACTCCAAAAACCGCATCACTGAAGAGACGCTGGCGAAATTACAGGATCTGGCGAAAGAGTGCGATCTGGCGGGCGCGATTAAGTCGATGTTCTCTGGCGAGAAGATCAACCGCACTGAAAACCGCGCCGTGCTGCACGTAGCGCTGCGTAACCGTAGCAATACCCCGATTTTGGTTGATGGCAAAGACGTAATGCCGGAAGTCAACGCGGTGCTGGAGAAGATGAAAACCTTCTCAGAAGCGATTATTTCCGGTGAGTGGAAAGGTTATACCGGCAAAGCAATCACTGACGTAGTGAACATCGGGATCGGCGGTTCTGACCTCGGCCCATACATGGTGACCGAAGCTCTGCGTCCGTACAAAAACCACCTGAACATGCACTTTGTTTCTAACGTCGATGGGACTCACATCGCGGAAGTGCTGAAAAAAGTAAACCCGGAAACCACGCTGTTCTTGGTAGCATCTAAAACCTTCACCACTCAGGAAACTATGACCAACGCCCATAGCGCGCGTGACTGGTTCCTGAAAGCGGCAGGTGATGAAAAACACGTTGCAAAACACTTTGCGGCGCTTTCCACCAATGCCAAAGCCGTTGGCGAGTTTGGTATTGATACTGCCAACATGTTCGAGTTCTGGGACTGGGTTGGCGGCCGTTACTCTTTGTGGTCAGCGATTGGCCTGTCGATTGTTCTCTCCATCGGCTTTGATAACTTCGTTGAACTGCTTTCCGGCGCACACGCGATGGACAAGCATTTCTCCACCACGCCTGCCGAGAAAAACCTGCCTGTACTGCTGGCGCTGATTGGCATCTGGTACAACAATTTCTTTGGTGCGGAAACTGAA

#### tRNA Metabolites

A table listing the "TranscribedGene" tRNA Metabolites is

In [33]:
df = TranscribedGene_Metabolites
trna_metabolites = df.loc[df['Metabolite Type'] == 'tRNA']
print("Number of tRNA metabolites = ",len(trna_metabolites))
trna_metabolites

Number of tRNA metabolites =  86


,Metabolite ID,Metabolite Name,Metabolite Formula,Metabolite Compartment,Metabolite Type
107,RNA_b0202,,C729H831N287O538P77,c,tRNA
108,RNA_b0203,,C722H823N291O530P76,c,tRNA
111,RNA_b0206,,C732H836N298O533P77,c,tRNA
114,RNA_b0216,,C732H836N298O533P77,c,tRNA
122,RNA_b0244,,C721H818N284O531P76,c,tRNA
217,RNA_b0536,,C731H832N292O539P77,c,tRNA
263,RNA_b0664,,C714H810N288O524P75,c,tRNA
264,RNA_b0665,,C714H810N288O524P75,c,tRNA
265,RNA_b0666,,C731H828N288O540P77,c,tRNA
266,RNA_b0668,,C714H810N288O523P75,c,tRNA


An example of the attributes of a "TranscribedGene" tRNA metabolite are

In [34]:
pd.DataFrame({i: str(v) for i, v in me.metabolites.RNA_b4163.__dict__.items() if not i.startswith('_') and v}, 
             index=['Attribute Values']).T

,Attribute Values
id,RNA_b4163
formula,C722H823N291O528P76
compartment,c
left_pos,4390382
right_pos,4390458
strand,+
RNA_type,tRNA
nucleotide_sequence,GCGGGAATAGCTCAGTTGGTAGAGCACGACCTTGCCAAGGTCGGGGTCGCGAGTTCGAGTCTCGTTTCCCGCTCCA


Reactions that consume or produce this tRNA metabolite.

In [35]:
me.metabolites.RNA_b4163.reactions

frozenset({<MEReaction DM_RNA_b4163 at 0x240690f8240>,
           <tRNAChargingReaction charging_tRNA_b4163_GGC at 0x2406907cfd0>,
           <tRNAChargingReaction charging_tRNA_b4163_GGU at 0x2406908a198>,
           <TranscriptionReaction transcription_TU00505_from_RpoD_mono at 0x240fa2887f0>})

#### rRNA Metabolites

A table listing the "TranscribedGene" rRNA metabolites is

In [36]:
df = TranscribedGene_Metabolites
rrna_metabolites = df.loc[df['Metabolite Type'] == 'rRNA']
print("Number of rRNA metabolites = ",len(rrna_metabolites))
rrna_metabolites

Number of rRNA metabolites =  22


,Metabolite ID,Metabolite Name,Metabolite Formula,Metabolite Compartment,Metabolite Type
106,RNA_b0201,,C14546H16574N5574O10866P1542,c,rRNA
109,RNA_b0204,,C27365H31180N10406O20498P2904,c,rRNA
110,RNA_b0205,,C1136H1297N449O843P120,c,rRNA
943,RNA_b2588,,C1136H1297N449O843P120,c,rRNA
944,RNA_b2589,,C27366H31182N10410O20499P2904,c,rRNA
946,RNA_b2591,,C14545H16576N5574O10866P1542,c,rRNA
1180,RNA_b3272,,C1135H1295N445O844P120,c,rRNA
1182,RNA_b3274,,C1136H1297N449O843P120,c,rRNA
1183,RNA_b3275,,C27365H31180N10406O20501P2904,c,rRNA
1186,RNA_b3278,,C14545H16574N5572O10869P1542,c,rRNA


An example of the attributes of a "TranscribedGene" rRNA metabolite are

In [37]:
pd.DataFrame({i: str(v) for i, v in me.metabolites.RNA_b4007.__dict__.items() if not i.startswith('_') and v}, 
             index=['Attribute Values']).T

,Attribute Values
id,RNA_b4007
formula,C14544H16573N5569O10869P1542
compartment,c
left_pos,4206169
right_pos,4207711
strand,+
RNA_type,rRNA
nucleotide_sequence,AAATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTCGAACGGTAACAGGAAGAAGCTTGCTTCTTTGCT...


Reactions that consume or produce this rRNA metabolite are

In [38]:
me.metabolites.RNA_b4007.reactions

frozenset({<MEReaction DM_RNA_b4007 at 0x240690df4e0>,
           <GenericFormationReaction RNA_b4007_to_generic_16s_rRNAs at 0x2410179c240>,
           <TranscriptionReaction transcription_TU0_1186_from_RpoD_mono at 0x240fa574710>,
           <TranscriptionReaction transcription_TU0_14487_from_RpoD_mono at 0x240fa25d320>})

#### ncRNA Metabolites

A table listing the "TranscribedGene" ncRNA metabolites can be found by

In [39]:
df = TranscribedGene_Metabolites
ncrna_metabolites = df.loc[df['Metabolite Type'] == 'ncRNA']
print("Number of ncRNA metabolites = ",len(ncrna_metabolites))
ncrna_metabolites

Number of ncRNA metabolites =  2


,Metabolite ID,Metabolite Name,Metabolite Formula,Metabolite Compartment,Metabolite Type
190,RNA_b0455,,C1081H1234N432O797P114,c,ncRNA
1128,RNA_b3123,,C3551H4060N1360O2669P377,c,ncRNA


An example of the attributes of a "TranscribedGene" ncRNA metabolite is shown below.

In [40]:
pd.DataFrame({i: str(v) for i, v in me.metabolites.RNA_b3123.__dict__.items() if not i.startswith('_') and v}, 
             index=['Attribute Values']).T

,Attribute Values
id,RNA_b3123
formula,C3551H4060N1360O2669P377
compartment,c
left_pos,3268237
right_pos,3268614
strand,-
RNA_type,ncRNA
nucleotide_sequence,GAAGCTGACCAGACAGTCGCCGCTTCGTCGTCGTCCTCTTCGGGGGAGACGGGCGGAGGGGAGGAAAGTCCGGGCTCCATAGGGCAGGGTGCCAGG...


Reactions that consume or produce this ncRNA metabolite are

In [41]:
me.metabolites.RNA_b3123.reactions

frozenset({<MEReaction DM_RNA_b3123 at 0x240690df5f8>,
           <ComplexFormation formation_RNase_P_cplx_mod_2:mg2 at 0x24069672e80>,
           <TranscriptionReaction transcription_TU482_with_TERM161_from_RpoD_mono at 0x240fa25d5f8>,
           <TranscriptionReaction transcription_TU543_with_TERM0281_from_RpoD_mono at 0x240fa5f7390>,
           <TranscriptionReaction transcription_TU543_with_TERM0282_from_RpoD_mono at 0x240fa5f7470>,
           <TranscriptionReaction transcription_TU543_with_TERM0283_from_RpoD_mono at 0x240fa5f7588>})

## "TranslatedGene" Metabolites

See section 4.3 "Translation Reactions" at https://cobrame.readthedocs.io/en/master/Reaction_Manipulation.html

### TranslatedGene (Protein) metabolite properties
For COBRAme ME-models, proteins are translated directly from mRNA metabolites not from TUs. This means that all information required to construct a TranslationReaction can be found in its TranslationData therefore no extra information is contained in a TranslatedGene object.

The attributes and methods associated with a TranslatedGene metabolite include (using "dir ()")

- amino_acid_sequence,
- annotation,
- charge,
- compartment,
- complexes,
- copy,
- elements,
- formula,
- formula_weight,
- id,
- metabolic_reactions,
- model,
- name,
- notes,
- reactions,
- remove_from_me_model,
- remove_from_model,
- summary,
- translation_data,
- y

The number of translated genes and their translated data objects can be found by

In [42]:
print('number of translation data objects = %i' %  len(list(me.translation_data)))
print('number of translated genes (proteins) = %i' %
      len([m.id for m in me.metabolites if type(m) == cobrame.TranslatedGene]))

number of translation data objects = 1569
number of translated genes (proteins) = 1569


Creating a table of the key "TranslatedGene" metabolites attributes

In [43]:
protein_type_metabolites = []
#translated_gene_metabolites = [for m in me.metabolites if type(m) == cobrame.TranslatedGene]
translated_gene_metabolites = [m.id for m in me.metabolites if type(m) == cobrame.TranslatedGene]
for m in translated_gene_metabolites: protein_type_metabolites.append(me.metabolites.get_by_id(m))

metabolite_ids = [m.id for m in protein_type_metabolites]
metabolite_formula = [m.formula for m in protein_type_metabolites]
metabolite_formula_weight = [m.formula_weight for m in protein_type_metabolites]
metabolite_compartment = [m.compartment for m in protein_type_metabolites]
metabolite_elements = [m.elements for m in protein_type_metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Type': metabolite_type,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Formula Weight': metabolite_formula_weight,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Elements': metabolite_elements,
                 }

TranslatedGene_Metabolites = pd.DataFrame(metaboliteList, columns= ['Metabolite ID','Metabolite Formula', 'Metabolite Formula Weight','Metabolite Compartment', 'Metabolite Elements'])
pd.set_option('display.max_rows', 500)
TranslatedGene_Metabolites[0:200]   

,Metabolite ID,Metabolite Formula,Metabolite Formula Weight,Metabolite Compartment,Metabolite Elements
0,protein_b0002,C3940H6306N1092O1188S35,89103.10624,c,"{'C': 3940, 'H': 6306, 'N': 1092, 'O': 1188, 'S': 35}"
1,protein_b0003,C1485H2345N413O439S19,33617.24750,c,"{'C': 1485, 'H': 2345, 'N': 413, 'O': 439, 'S': 19}"
2,protein_b0004,C2120H3311N564O626S12,47100.15654,c,"{'C': 2120, 'H': 3311, 'N': 564, 'O': 626, 'S': 12}"
3,protein_b0007,C2391H3689N602O619S28,51669.35636,c,"{'C': 2391, 'H': 3689, 'N': 602, 'O': 619, 'S': 28}"
4,protein_b0008,C1563H2500N420O480S7,35079.55510,c,"{'C': 1563, 'H': 2500, 'N': 420, 'O': 480, 'S': 7}"
5,protein_b0009,C937H1515N256O289S7,21215.05180,c,"{'C': 937, 'H': 1515, 'N': 256, 'O': 289, 'S': 7}"
6,protein_b0014,C2996H4864N842O987S15,68952.70156,c,"{'C': 2996, 'H': 4864, 'N': 842, 'O': 987, 'S': 15}"
7,protein_b0015,C1779H2819N533O549S16,40970.69986,c,"{'C': 1779, 'H': 2819, 'N': 533, 'O': 549, 'S': 16}"
8,protein_b0019,C1937H3109N480O487S14,41362.24516,c,"{'C': 1937, 'H': 3109, 'N': 480, 'O': 487, 'S': 14}"
9,protein_b0023,C414H722N138O115S3,9569.21308,c,"{'C': 414, 'H': 722, 'N': 138, 'O': 115, 'S': 3}"


The amino acid sequence of a given metabolite can be found by

In [44]:
me.metabolites.protein_b0002.amino_acid_sequence

'MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNHLVAMIEKTISGQDALPNISDAERIFAELLTGLAAAQPGFPLAQLKTFVDQEFAQIKHVLHGISLLGQCPDSINAALICRGEKMSIAIMAGVLEARGHNVTVIDPVEKLLAVGHYLESTVDIAESTRRIAASRIPADHMVLMAGFTAGNEKGELVVLGRNGSDYSAAVLAACLRADCCEIWTDVDGVYTCDPRQVPDARLLKSMSYQEAMELSYFGAKVLHPRTITPIAQFQIPCLIKNTGNPQAPGTLIGASRDEDELPVKGISNLNNMAMFSVSGPGMKGMVGMAARVFAAMSRARISVVLITQSSSEYSISFCVPQSDCVRAERAMQEEFYLELKEGLLEPLAVTERLAIISVVGDGMRTLRGISAKFFAALARANINIVAIAQGSSERSISVVVNNDDATTGVRVTHQMLFNTDQVIEVFVIGVGGVGGALLEQLKRQQSWLKNKHIDLRVCGVANSKALLTNVHGLNLENWQEELAQAKEPFNLGRLIRLVKEYHLLNPVIVDCTSSQAVADQYADFLREGFHVVTPNKKANTSSMDYYHQLRYAAEKSRRKFLYDTNVGAGLPVIENLQNLLNAGDELMKFSGILSGSLSYIFGKLDEGMSFSEATTLAREMGYTEPDPRDDLSGMDVARKLLILARETGRELELADIEIEPVLPAEFNAEGDVAAFMANLSQLDDLFAARVAKARDEGKVLRYVGNIDEDGVCRVKIAEVDGNDPLFKVKNGENALAFYSHYYQPLPLVLRGYGAGNDVTAAGVFADLLRTLSWKLGV'

The reactions that produce or consume a given metabolite are

In [45]:
me.metabolites.protein_b0002.reactions

frozenset({<ComplexFormation formation_ASPKINIHOMOSERDEHYDROGI-CPLX at 0x24069510d30>,
           <TranslationReaction translation_b0002 at 0x240fa0de940>})

The metabolic reactions that a specific protein helps catalyze are

In [46]:
me.metabolites.protein_b0002.metabolic_reactions

[<MetabolicReaction ASPK_FWD_ASPKINIHOMOSERDEHYDROGI-CPLX at 0x240699bd160>,
 <MetabolicReaction HSDy_FWD_ASPKINIHOMOSERDEHYDROGI-CPLX at 0x24069fd1390>,
 <MetabolicReaction HSDy_REV_ASPKINIHOMOSERDEHYDROGI-CPLX at 0x24069fd12b0>,
 <MetabolicReaction ASPK_REV_ASPKINIHOMOSERDEHYDROGI-CPLX at 0x240699aafd0>]

The complexes associated with a specific protein are

In [47]:
me.metabolites.protein_b0002.complexes

[<Complex ASPKINIHOMOSERDEHYDROGI-CPLX at 0x24069510d68>]

The nucleotide sequence is kept in the translation data

In [48]:
me.metabolites.protein_b0002.translation_data.nucleotide_sequence

'ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTTGACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGCTGATTTGCCGTGGCGAGAAAATGTCGATCGCCATTATGGCCGGCGTATTAGAAGCGCGCGGTCACAACGTTACTGTTATCGATCCGGTCGAAAAACTGCTGGCAGTGGGGCATTACCTCGAATCTACCGTCGATATTGCTGAGTCCACCCGCCGTATTGCGGCAAGCCGCATTCCGGCTGATCACATGGTGCTGATGGCAGGTTTCACCGCCGGTAATGAAAAAGGCGAACTGGTGGTGCTTGGACGCAACGGTTCCGACTACTCTGCTGCGGTGCTGGCTGCCTGTTTACGCGCCGATTGTTGCGAGATTTGGACGGACGTTGACGGGGTCTATACCTGCGACCCGCGTCAGGTGCCCGATGCGAGGTTGTTGAAGTCGATGTCCTACCAGGAAGCGATGGAGCTTTCCTACTTCGGCGCTAAAGTTCTTCACCCCCGCACCATTACCCCCATCGCCCAGTTCCAGATCCCTTGCCTGATTAAAAATACCGGAAATCCTCAAGCACCAGGTACGCTCATTGGTGCCAGCCGTGATGAAGACGAATTACCGGTCAAGGGCATTTCCAATCTGAATAACATGGCAATGTTCAGCGTTTCTGGTCCGGGGATGAAAGGGATGGTCGGCATGGCGGCG

The "translation_data" stoes the following methods and information

- add_elongation_subreactions,
- add_initiation_subreactions,
- add_termination_subreactions,
- amino_acid_count,
- amino_acid_sequence,
- codon_count,
- first_codon,
- id,
- last_codon,
- mRNA,
- model,
- nucleotide_sequence,
- parent_reactions,
- protein,
- subreactions,
- subreactions_from_sequence,
- update_parent_reactions

In [49]:
dir(me.metabolites.protein_b0002.translation_data)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_itercodons',
 '_model',
 '_parent_reactions',
 'add_elongation_subreactions',
 'add_initiation_subreactions',
 'add_termination_subreactions',
 'amino_acid_count',
 'amino_acid_sequence',
 'codon_count',
 'first_codon',
 'id',
 'last_codon',
 'mRNA',
 'model',
 'nucleotide_sequence',
 'parent_reactions',
 'protein',
 'subreactions',
 'subreactions_from_sequence',
 'update_parent_reactions']

In [50]:
me.metabolites.protein_b0002.translation_data.mRNA

'RNA_b0002'

In [51]:
me.metabolites.protein_b0002.translation_data.amino_acid_count

defaultdict(int,
            {'met__L_c': 23,
             'arg__L_c': 46,
             'val__L_c': 69,
             'leu__L_c': 89,
             'lys__L_c': 34,
             'phe__L_c': 30,
             'gly_c': 63,
             'thr__L_c': 34,
             'ser__L_c': 51,
             'ala__L_c': 91,
             'asn__L_c': 38,
             'glu__L_c': 53,
             'asp__L_c': 44,
             'ile__L_c': 46,
             'gln__L_c': 30,
             'pro__L_c': 29,
             'his__L_c': 14,
             'cys__L_c': 12,
             'tyr__L_c': 20,
             'trp__L_c': 4})

In [52]:
me.metabolites.protein_b0002.translation_data.codon_count

defaultdict(int,
            {'AUG': 23,
             'CGA': 3,
             'GUG': 27,
             'UUG': 13,
             'AAG': 12,
             'UUC': 19,
             'GGC': 22,
             'GGU': 22,
             'ACA': 2,
             'UCA': 6,
             'GCA': 14,
             'AAU': 22,
             'GAA': 40,
             'CGU': 18,
             'UUU': 11,
             'CUG': 43,
             'GUU': 19,
             'GCC': 36,
             'GAU': 30,
             'AUU': 30,
             'AGC': 12,
             'AGG': 2,
             'CAG': 19,
             'GGG': 10,
             'ACC': 19,
             'GUC': 18,
             'CUC': 13,
             'UCU': 11,
             'CCC': 6,
             'AAA': 22,
             'AUC': 15,
             'AAC': 16,
             'CAC': 6,
             'GCG': 26,
             'GCU': 15,
             'UUA': 10,
             'CUU': 8,
             'ACG': 8,
             'GGA': 9,
             'CCG': 18,
             'CAA': 11,
        